In [ ]:
from PIL import Image
import dask.array as da
from matplotlib import colormaps as cm

In [ ]:
"""
embryo shape (t,c,z,y,x): [161, 1, 116, 6440, 6440]
tissue shape (t,c,z,y,x): [1, 9, 80, 10807, 9890],
"""

dataset_path = {
    'embryo': '../data/embryo.ome.zarr/0',
    'tissue-on': '../data/birefringent-data.ome.zarr/0',
    'tissue-off': '../data/birefringent-data.ome.zarr/0'
}
tcz_slice_axis = [(80, 0, 55), (0, 0, 40), (0, 1, 40)]
xy_crop_regions = [((5,-5),(5,-5)), ((540,-540),(200,-200)), ((540,-540),(200,-200))]
cranges = [(1.33, 1.36), (2.2,2.3), (-0.002,0.002)]
cmap_list = ['gray', 'gray', 'PiYG']

In [ ]:
def array_to_img(image,cmap_name,vmin,vmax):
    norm_img = (image-vmin)/(vmax-vmin)
    color_img = cm.get_cmap(cmap_name)(norm_img)
    color_img = color_img[:,:,:-1]*255
    return Image.fromarray(color_img.astype('u1'), mode='RGB')

In [ ]:
for (data_key,data_path), (t,c,z), (y_crop, x_crop), crange, cmap_name in zip(dataset_path.items(), tcz_slice_axis, xy_crop_regions, cranges, cmap_list):
    data = da.from_zarr(data_path, mode='r')[t,c,:,slice(*y_crop),slice(*x_crop)]
    slice_dataset = {
        'zslice': data[z,:,:],
        'yupper-slice': data[:,0,:],
        'ylower-slice': data[:,-1,:],
        'xupper-slice': data[:,:,0],
        'xlower-slice': data[:,:,-1],
    }
    for slice_key, slice_data in slice_dataset.items():
        save_path = f'../figure/slice-image/{data_key}-{slice_key}.png'
        result = array_to_img(slice_data.compute(), cmap_name, *crange)
        result.save(save_path)